In [1]:
import sys
utils = '/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/utils'
sys.path.insert(0, utils)

In [4]:
from pathlib import Path
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sql import connect
# from ml import models

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# import tensorflow as tf
import HydroErr as he

In [14]:
proj_dir = Path('../')

## Data Preparation

In [5]:
# Load dataset from the SQL database
def load_sql_dataset(query, connection, columns, chunksize = 1000):
    """Load dataset from the SQL database"""
    df = pd.read_sql(query, connection, columns=columns, parse_dates=['date'], chunksize=chunksize)
    return df


In [6]:
# Create a connection object to the MySQL database
conn = connect.Connect('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Methods/database_management/mysql_config.ini')
cursor = conn.conn.cursor()

Connecting to MySQL database...
Connection established.


In [7]:
# Reaches of interest to CRITFC
reaches_of_interest = ["Yakima_River_51","White_Salmon_River_11","Snake_River_186","Snake_River_161","Little_White_Salmon_River_5","Hood_River_3","Deschutes_River_54","Cowlitz_River_27","Columbia_River_255","Columbia_River_237","Columbia_River_235","Columbia_River_233","Columbia_River_232","Columbia_River_231","Columbia_River_229","Columbia_River_226","Columbia_River_223","Columbia_River_221","Columbia_River_203","Columbia_River_195","Columbia_River_193","Columbia_River_178","Columbia_River_174","Columbia_River_165","Columbia_River_160","Columbia_River_150","Columbia_River_143","Columbia_River_131","Columbia_River_121"]
# print(tuple(reaches_of_interest))

In [19]:
# # data = load_sql_dataset(query = f"""
# # SELECT
# # 	ReachLandsatWaterTemp.date,
# #     ReachLandsatWaterTemp.Value as watertemp,
# #     ReachLandsatLandTemp.Value as landtemp,
# #     ReachNDVI.Value as NDVI,
# #     Reaches.ClimateClass
# # FROM
# # 	ReachLandsatWaterTemp
# # INNER JOIN ReachLandsatLandTemp USING (date, ReachID)
# # INNER JOIN ReachNDVI USING (date, ReachID)
# # INNER JOIN Reaches USING (ReachID)
# # WHERE Reaches.Name NOT IN {tuple(reaches_of_interest)} AND ReachLandsatWaterTemp.Value > 0
# # ORDER BY RAND();
# # """, connection = conn.engine, columns = ['date', 'watertemp', 'landtemp', 'NDVI', 'ClimateClass'])

# query = f"""
# SELECT 
#     STR_TO_DATE(CONCAT(Year,
#                     '-',
#                     LPAD(Month, 2, '00'),
#                     '-',
#                     LPAD(DayOfMonth, 2, '00')),
#             '%Y-%m-%d') AS Date,
#     Month,
#     DayOfMonth,
#     ROUND(WaterTemp, 2) as WaterTemp,
#     ROUND(LandTemp, 2) as LandTemp,
#     ROUND(NDVI, 2) as NDVI,
#     ClimateClass,
#     --     ROUND(((watertemp - WaterTemperature) / WaterTemperature),
#     --             2) AS PercentDeviation,
#     --     ROUND((watertemp - WaterTemperature), 2) AS Deviation,
#     Width,
#     ReachID,
#     ROUND(InsituTemp, 2) AS InsituTemp
# FROM
#     (SELECT 
#         1 AS DayOfMonth,
#             MONTH(ReachLandsatWaterTemp.date) AS Month,
#             YEAR(ReachLandsatWaterTemp.date) AS Year,
#             AVG(ReachLandsatWaterTemp.Value) AS WaterTemp,
#             AVG(ReachLandsatLandTemp.Value) AS LandTemp,
#             AVG(ReachNDVI.Value) AS NDVI,
#             IFNULL(Reaches.WidthMean, 30) AS Width,
#             Reaches.ClimateClass AS ClimateClass,
#             ReachLandsatWaterTemp.ReachID AS ReachID
#     FROM
#         ReachLandsatWaterTemp
#     INNER JOIN ReachLandsatLandTemp USING (date , ReachID)
#     INNER JOIN ReachNDVI USING (date , ReachID)
#     INNER JOIN Reaches USING (ReachID)
#     WHERE
#         ReachLandsatWaterTemp.Value > 0
#     --    Reaches.Name NOT IN {tuple(reaches_of_interest)}
#     --        AND ReachLandsatWaterTemp.Value > 0
#     GROUP BY DayOfMonth , Month , Year , ClimateClass , ReachID , Width) AS T
#     --         INNER JOIN
#     --     ReachLandsatLTMSemiMonthly USING (DayOfMonth , Month , ReachID)
#         LEFT JOIN
#     (SELECT 
#         1 AS DayOfMonth,
#             MONTH(ReachInsituWaterTemp.date) AS Month,
#             YEAR(ReachInsituWaterTemp.date) AS Year,
#             AVG(ReachInsituWaterTemp.Value) AS InsituTemp,
#             ReachInsituWaterTemp.ReachID AS ReachID
#     FROM
#         ReachInsituWaterTemp
#     INNER JOIN Reaches USING (ReachID)
#     WHERE
#         ReachInsituWaterTemp.Value > 0
#     GROUP BY DayOfMonth , Month , Year , ReachID) AS I USING (DayOfMonth , Month , Year , ReachID)
# ORDER BY RAND();
# """ 
# # print(query)

# df = conn.query_with_fetchmany(query, chunksize=100)


df = pd.read_csv(proj_dir / "Methods/2.Data/ML_input_data.csv")

In [20]:
# df = pd.concat([chunk for chunk in data], ignore_index=True)
df['Date'] = pd.to_datetime(df['Date'])
df['DayOfYear'] = df['Date'].dt.dayofyear

In [21]:
df_with_insitu = df[df["InsituTemp"].notnull()].copy()
df_with_insitu["deviation"] = df_with_insitu["WaterTemp"] - df_with_insitu["InsituTemp"]

In [22]:
import numpy as np
from scipy import stats

In [23]:
print("var > 120 m: " + str(df_with_insitu[df_with_insitu["Width"] > 120]["deviation"].var()) + "\n var < 120 m: " + str(df_with_insitu[df_with_insitu["Width"] < 120]["deviation"].var()))

var > 120 m: 18.709935246308522
 var < 120 m: 52.43550017608853


In [24]:
stats.ttest_ind(
    list(df_with_insitu[df_with_insitu["Width"] > 120]["deviation"].to_numpy()),
    list(df_with_insitu[df_with_insitu["Width"] < 120]["deviation"].to_numpy()),
    alternative="less",
    equal_var=False,
)

Ttest_indResult(statistic=-28.49008498695906, pvalue=2.2733504793965857e-158)

In [25]:
df_with_insitu['deviation'] = pd.to_numeric(df_with_insitu['deviation'])

In [26]:
df_with_insitu[df_with_insitu["Width"] >120].describe()

,Month,DayOfMonth,WaterTemp,LandTemp,NDVI,ClimateClass,Width,ReachID,InsituTemp,DayOfYear,deviation
count,5597.000000,5597.000000,5597.000000,5597.000000,5597.000000,5597.000000,5597.000000,5597.000000,5597.000000,5597.000000,5597.000000
mean,6.611756,8.368948,15.938932,23.438351,0.388185,10.823834,775.432196,713.188137,13.129346,178.267286,2.809585
std,2.853066,6.990895,8.727840,12.958836,0.149866,7.358169,416.215967,442.835397,6.047180,86.793481,4.325498
min,1.000000,1.000000,-18.600000,-20.350000,-0.200000,5.000000,155.000000,203.000000,0.100000,1.000000,-22.580000
25%,4.000000,1.000000,9.220000,14.170000,0.280000,7.000000,498.000000,260.000000,7.860000,105.000000,0.060000
50%,7.000000,15.000000,17.150000,24.940000,0.350000,7.000000,655.000000,634.000000,13.720000,182.000000,3.030000
75%,9.000000,15.000000,23.150000,33.260000,0.480000,9.000000,1094.000000,1175.000000,18.280000,244.000000,5.730000
max,12.000000,15.000000,37.640000,50.860000,0.970000,27.000000,1996.000000,1289.000000,28.760000,350.000000,16.930000


In [27]:
df_with_insitu[df_with_insitu["Width"] <120].describe()

,Month,DayOfMonth,WaterTemp,LandTemp,NDVI,ClimateClass,Width,ReachID,InsituTemp,DayOfYear,deviation
count,2286.000000,2286.000000,2286.000000,2286.000000,2286.000000,2286.000000,2286.000000,2286.000000,2286.000000,2286.000000,2286.000000
mean,6.747594,8.263342,20.754939,23.739497,0.478920,7.997813,52.618985,748.790026,13.326750,182.292651,7.428189
std,2.878834,6.996575,11.277471,12.080496,0.122506,3.133568,21.064547,437.992359,6.000004,87.508282,7.241236
min,1.000000,1.000000,-19.390000,-16.940000,0.020000,5.000000,30.000000,12.000000,0.100000,1.000000,-20.020000
25%,4.000000,1.000000,13.182500,15.420000,0.380000,7.000000,30.000000,577.000000,8.692500,106.000000,2.592500
50%,7.000000,15.000000,21.800000,25.220000,0.480000,7.000000,50.000000,789.000000,13.210000,182.000000,7.055000
75%,9.000000,15.000000,28.530000,32.805000,0.570000,9.000000,72.000000,1327.000000,18.297500,245.000000,11.360000
max,12.000000,15.000000,56.120000,50.340000,0.920000,27.000000,97.000000,1343.000000,29.050000,350.000000,34.070000
